In [1]:
!rm -rf /kaggle/working/*

In [2]:
!git clone https://github.com/windymv025/VALL_E_X.git
!mv -v /kaggle/working/VALL_E_X/* /kaggle/working/
!rm -rf VALL_E_X

Cloning into 'VALL_E_X'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 230 (delta 47), reused 214 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (230/230), 15.25 MiB | 26.12 MiB/s, done.
Resolving deltas: 100% (47/47), done.
renamed '/kaggle/working/VALL_E_X/LICENSE' -> '/kaggle/working/LICENSE'
renamed '/kaggle/working/VALL_E_X/README.md' -> '/kaggle/working/README.md'
renamed '/kaggle/working/VALL_E_X/customs' -> '/kaggle/working/customs'
renamed '/kaggle/working/VALL_E_X/data' -> '/kaggle/working/data'
renamed '/kaggle/working/VALL_E_X/descriptions.py' -> '/kaggle/working/descriptions.py'
renamed '/kaggle/working/VALL_E_X/examples.py' -> '/kaggle/working/examples.py'
renamed '/kaggle/working/VALL_E_X/exp' -> '/kaggle/working/exp'
renamed '/kaggle/working/VALL_E_X/images' -> '/kaggle/working/images'
renamed '/kaggle/working/VALL_E_X/infer.ipynb' -> '/kaggle/working

In [3]:
!pip install -q -r requirements.txt

In [4]:
input_dataset_path = '/kaggle/input/vivos-vietnamese-speech-dataset-ljspeech-format/vivos'
input_train_path = f'{input_dataset_path}/train'
input_test_path = f'{input_dataset_path}/test'
prompt_file_name = 'audio_ann_sum.txt'

# dataset_path = 'vivos_datasets'
# train_path = f'{dataset_path}/train'
# test_path = f'{dataset_path}/test'

In [5]:
!mkdir -p $train_path
!mkdir -p $test_path

mkdir: missing operand
Try 'mkdir --help' for more information.
mkdir: missing operand
Try 'mkdir --help' for more information.


In [6]:
import shutil
import os
import h5py
import glob
import torch
import torchaudio

# import torch_xla
# import torch_xla.core.xla_model as xm

# print(f'PyTorch can access {xm.xla_device()} TPU cores')
# tpu_device = xm.xla_device()

# torch_xla.device_count()

In [7]:
!pip install -q gdown
!gdown 10gdQWvP-K_e1undkvv0p2b7SU6I4Egyl

Downloading...
From (original): https://drive.google.com/uc?id=10gdQWvP-K_e1undkvv0p2b7SU6I4Egyl
From (redirected): https://drive.google.com/uc?id=10gdQWvP-K_e1undkvv0p2b7SU6I4Egyl&confirm=t&uuid=da3af224-2df8-4b61-b5f0-74a8f0598aef
To: /kaggle/working/vallex-checkpoint.pt
100%|██████████████████████████████████████| 1.48G/1.48G [00:16<00:00, 89.8MB/s]


In [8]:
!mkdir -p checkpoints
!cp vallex-checkpoint.pt checkpoints/vallex-checkpoint_modified.pt
!mv vallex-checkpoint.pt checkpoints/vallex-checkpoint.pt

In [9]:
# !python -X utf8 launch-ui.py
# --keep-last-k 2 \

In [10]:
!python3 train.py \
    --dtype "bfloat16" \
    --num-epochs 100 \
    --save-every-n 10 \
    --world-size 1 \
    --keep-last-k 2 \
    --visualize True \
    --train_dir $input_train_path \
    --valid_dir $input_test_path 

Operating System: Linux
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|███████████████████████████████████████| 88.9M/88.9M [00:00<00:00, 179MB/s]
2024-10-18 11:50:35,401 INFO [train.py:861] Training started
2024-10-18 11:50:35,402 INFO [train.py:880] Device: cuda:0
2024-10-18 11:50:35,403 INFO [train.py:881] {'best_train_loss': inf, 'best_valid_loss': inf, 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 100, 'reset_interval': 200, 'valid_interval': 10000, 'world_size': 1, 'master_port': 12354, 'tensorboard': True, 'num_epochs': 100, 'start_epoch': 1, 'start_batch': 0, 'exp_dir': PosixPath('exp/valle_dev'), 'optimizer_name': 'ScaledAdam', 'scheduler_name': 'Eden', 'base_lr': 0.005, 'warmup_steps': 200, 'seed': 42, 'inf_check': False, 'save_every_n': 10, 'keep_last_k': 2, 'average_period': 0, 'accumulate_grad_steps': 1, 'dtype': 'bfloat16', 'filt

In [11]:
!cp /kaggle/working/exp/valle_dev/best-train-loss.pt /checkpoints/vallex-checkpoint.pt

cp: cannot stat '/kaggle/working/exp/valle_dev/best-train-loss.pt': No such file or directory


In [12]:
from utils.generation import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

# download and load all models
preload_models()

# generate audio from text
text_prompt = """
Xin chao, Viet Nam.
"""
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("vallex_generation.wav", SAMPLE_RATE, audio_array)

# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)

/kaggle/working/utils/generation.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(checkpoints_dir, model_checkpoint_name), map_locat

VALL-E EOS [0 -> 102]
